In [ ]:
from pyalgotrade.bitstamp import barfeed
from pyalgotrade.bitstamp import broker
from pyalgotrade import strategy

class Strategy(strategy.BaseStrategy):
    def __init__(self, feed, brk):
        super(Strategy, self).__init__(feed, brk)
        self.__tack_instrument = "BTC"
        self.__trade_instrument = "ETH"
        self.__prices = feed[self.__tack_instrument].getPriceDataSeries()
        self.__position = None
        self.__posSize = 0.5

        # Subscribe to order book update events to get prices.
        feed.getOrderBookUpdateEvent().subscribe(self.__onOrderBookUpdate)
        
    def __onOrderBookUpdate(self, orderBookUpdate):
        close = orderBookUpdate.getClosePrices()[0]
        open_ = orderBookUpdate.getOpenPrices()[0]

        if close != self.__close or open_ != self.__open_:
            self.__close = close
            self.__open_ = open_
            self.info("Order book updated. Open: %s. Closed: %s" % (self.__open_, self.__close))
            
    def onEnterOk(self, position):
        self.info("Position opened at %s" % (position.getEntryOrder().getExecutionInfo().getPrice()))

    def onEnterCanceled(self, position):
        self.info("Position entry canceled")
        self.__position = None
        
    def onBars(self, bars):
        track_bar = bars[self.__track_instrument]
        self.info("BTC price: %s. BTC volume: %s." % \
                  (track_bar.getClose(), track_bar.getVolume()))
        
        # Wait until we get the prices.
        if self.__open_ is None:
            return

        # If a position was not opened, check if we should enter a position.
        if self.__position is None:
            price_change = self.__open_ / self.__close
            # If BTC moves 5% down we go long
            if price_change < 0.95:
                self.info("Long signal.")
                self.__position = self.enterLong(self.__trade_instrument, self.__posSize, True)
            # If BTC moves 5% up we go short    
            elif price_change < 1.05:
                self.info("Short signal.")
                self.__position = self.enterShort(self.__trade_instrument, self.__posSize, True)

In [ ]:
barFeed = barfeed.LiveTradeFeed()
brk = broker.PaperTradingBroker(1000, barFeed)
strat = Strategy(barFeed, brk)

strat.run()